In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords

In [2]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [3]:
url="https://en.wikipedia.org/wiki/List_of_Formula_One_World_Drivers%27_Champions"

In [4]:
try:
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    response.raise_for_status()  # Handle errors
except requests.exceptions.RequestException as e:
    print("Error fetching the webpage:", e)
    exit()

In [5]:
soup = BeautifulSoup(response.text, "html.parser")

In [6]:
text = soup.get_text(separator=" ")

In [7]:
words = re.findall(r"\b\w{3,}\b", text.lower())

In [8]:
stop_words = set(stopwords.words("english"))
filtered_words = [word for word in words if word not in stop_words]

In [9]:
word_counts = Counter(filtered_words)

In [10]:
common_words = word_counts.most_common(100)

In [11]:
conn = sqlite3.connect("seo_dictionary.db")
cursor = conn.cursor()

In [12]:
cursor.execute("""CREATE TABLE IF NOT EXISTS seo_words (
        word TEXT PRIMARY KEY,
        frequency INTEGER,
        last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP)""")

In [13]:
for word, freq in common_words:
    cursor.execute("""INSERT INTO seo_words (word, frequency) 
        VALUES (?, ?) 
        ON CONFLICT(word) DO UPDATE SET frequency = frequency + ?
    """, (word, freq, freq))

conn.commit()

In [14]:
cursor.execute("SELECT * FROM seo_words ORDER BY frequency DESC LIMIT 20")
top_words = cursor.fetchall()
conn.close()

In [15]:
print("\nTop SEO Keywords from the F1 Champions Page:")
for word, freq, _ in top_words:
    print(f"{word}: {freq}")


Top SEO Keywords from the F1 Champions Page:
round: 75
2016: 74
drivers: 66
retrieved: 66
archived: 62
original: 62
world: 50
2020: 50
november: 47
championship: 44
com: 39
ferrari: 37
formula1: 37
formula: 35
one: 31
mercedes: 31
schumacher: 30
hamilton: 29
july: 29
season: 28
